In [ ]:
cur_el = joblib.load('it.sav')

In [ ]:
base_folder = Path('C:/Users/ilkele4s/')


main_results_folder = Path('ML_Results/SVM_stratified_singleElectrode_results')
new_results_folder = 'electrode_' +str(cur_el)

data_folder = Path('Data/Electrodes')
data_file = 'electrode_' +str(cur_el) + '.csv'

cur_electrode_name = 'electrode_' +str(cur_el)

In [ ]:
results_dir = os.path.join(base_folder,main_results_folder)

#if cur_el == 1:
#    os.mkdir(results_dir)

new_results_dir = os.path.join(results_dir,new_results_folder)

In [ ]:
if not os.path.exists(new_results_dir): # if the directory does not exist

    os.mkdir(new_results_dir)


print(new_results_dir)

In [ ]:
# load the data 
data_dir = os.path.join(base_folder,data_folder)

data_file_path = os.path.join(data_dir,data_file)

print(data_file_path)

Data = pd.read_csv(data_file_path, header = None)

In [ ]:
np.shape(Data)

In [ ]:
# create a label set

n_zones = 45
n_trials = 32

label_set = np.r_[1:n_zones+1]
labels = np.tile(label_set, 32)


In [ ]:
# merge samples and labels

Data[300] = labels

In [ ]:
# remove NAN values

Data.dropna(inplace=True)
Data = Data.reset_index()
del Data['index']

In [ ]:
Data_shape_noNan = np.shape(Data)
print(Data_shape_noNan)

In [ ]:
labels_rem = np.array(Data[300])

In [ ]:
#np.shape(labels_rem)

In [ ]:
#print(labels_rem)

In [ ]:
samples = np.array(Data.drop(Data.columns[300],axis = 1))

In [ ]:
# train - test split

train_samples, test_samples, train_labels, test_labels = train_test_split(samples, labels_rem, test_size=0.20, stratify=labels_rem, random_state=42)

In [ ]:
# standardization
data_scale = 1
if data_scale == 1:

    sc = StandardScaler()
    train_samples = sc.fit_transform(train_samples)
    test_samples = sc.transform(test_samples)

In [ ]:
comparison_matrix = train_samples[:, np.newaxis] == test_samples
true_ind = np.where(comparison_matrix == True)
tekrar = np.shape(true_ind)

if not tekrar[1] == 0 :
    print('Repeated elements!')
else :
     print('No repeated elements!')
          

In [ ]:
#plt.figure(figsize=(15, 10))
#plt.hist(train_labels, bins=np.arange(47) - 0.5, linewidth=1, edgecolor="white")

In [ ]:
#plt.figure(figsize=(15, 10))
#plt.hist(test_labels, bins=np.arange(47) - 0.5, linewidth=1, edgecolor="white")

In [ ]:
# model training with default parameters

svm_model_base = svm.SVC()
svm_model_base.fit(train_samples,train_labels)

In [ ]:
# predictions with
prediction_base = svm_model_base.predict(test_samples)

In [ ]:
joblib.dump(prediction_base, os.path.join(new_results_dir,'base_prediction.sav'))

In [ ]:
# creating grid

param_grid = [{'class_weight': [None, 'balanced'], 
               'C':[0.5, 1, 10, 100], 
               'gamma': ['scale', 1, 0.1, 0.01, 0.001],
               'kernel': ['rbf', 'poly', 'sigmoid','linear'], 
               'decision_function_shape': ['ovo','ovr']}]

In [ ]:
# grid search

optimal_params = GridSearchCV(svm.SVC(), param_grid, cv=5, scoring='balanced_accuracy', verbose=0)

optimization = optimal_params.fit(train_samples, train_labels)


In [ ]:
#print(optimal_params.best_params_)

In [ ]:
joblib.dump(optimal_params.best_params_, os.path.join(new_results_dir,'best_params.sav'))

In [ ]:
dict_string = "\n".join([f"{key}: {value}" for key, value in optimal_params.best_params_.items()])

In [ ]:
# getting optimal values for the parameters

C_ = optimal_params.best_params_.get('C')
dfs_ = optimal_params.best_params_.get('decision_function_shape')
gamma_ = optimal_params.best_params_.get('gamma')
kernel_ = optimal_params.best_params_.get('kernel')
weight_ = optimal_params.best_params_.get('class_weight')

In [ ]:
# train the model with the optimal values
svm_model_optimized = svm.SVC(C=C_, class_weight=weight_, decision_function_shape=dfs_, gamma=gamma_, kernel=kernel_)
svm_model_optimized.fit(train_samples,train_labels)

In [ ]:
prediction_optimized = svm_model_optimized.predict(test_samples)

In [ ]:
joblib.dump(prediction_optimized, os.path.join(new_results_dir,'optimized_prediction.sav'))

In [ ]:
figure_name1 = cur_electrode_name + '_Cm_base.png'

cm = confusion_matrix(test_labels, prediction_base, normalize='true', labels=svm_model_base.classes_)

plt.figure(figsize=(20, 16))
sns.heatmap(cm, fmt='.2', annot=True,annot_kws={"size": 8}, xticklabels=svm_model_base.classes_, yticklabels=svm_model_base.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')


plt.savefig(os.path.join(new_results_dir,figure_name1))
#plt.show()


In [ ]:
Acc_base= sum(test_labels == prediction_base)/len(test_labels)*100
print('Acc_base is',Acc_base)

In [ ]:
Acc_base_balanced = balanced_accuracy_score(test_labels, prediction_base)
print('Acc_base_balanced is', Acc_base_balanced)

In [ ]:
figure_name2 = cur_electrode_name + '_Cm_optim.png'

cm = confusion_matrix(test_labels, prediction_optimized, normalize='true', labels=svm_model_base.classes_)

plt.figure(figsize=(20, 16))
sns.heatmap(cm, fmt='.2', annot=True, annot_kws={"size": 8}, xticklabels=svm_model_base.classes_, yticklabels=svm_model_base.classes_)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')


plt.savefig(os.path.join(new_results_dir,figure_name2))
#plt.show()


In [ ]:
Acc_optimized= sum(test_labels == prediction_optimized)/len(test_labels)*100
print('Acc_optimized is', Acc_optimized)

In [ ]:
Acc_optimized_balanced = balanced_accuracy_score(test_labels, prediction_optimized)
print('Acc_optimized_balanced is',Acc_optimized_balanced)

In [ ]:
f1_base = f1_score(test_labels, prediction_base, average='weighted')
print('f1_base score is', f1_base)

In [ ]:
f1_optimized = f1_score(test_labels, prediction_optimized, average='weighted')
print('f1_optimized score is', f1_optimized)

In [40]:
file_path = os.path.join(results_dir,'best_params.txt')

with open(file_path, 'w') as file1:
   
          file1.write(cur_electrode_name + ':' + dict_string+'\n')

In [41]:
file_path2 = os.path.join(results_dir,'Acc_base.txt')

with open(file_path2, 'w') as file2:
    
    file2.write(cur_electrode_name + ':' + str(Acc_base)+'\n')

In [28]:
file_path3 = os.path.join(results_dir, 'Acc_base_balanced.txt')

with open(file_path3, 'w') as file3:
    
    file3.write(cur_electrode_name + ':' + str(Acc_base_balanced)+'\n')


In [ ]:
file_path4 = os.path.join(results_dir, 'Acc_optimized.txt')

with open(file_path4, 'w') as file4:
    
    file4.write(cur_electrode_name + ':' + str(Acc_optimized)+'\r\n')

In [ ]:
file_path5 = os.path.join(results_dir, 'Acc_optimized_balanced.txt')

with open(file_path5, 'w') as file5:
    
    file5.write(cur_electrode_name + ':' + str(Acc_optimized_balanced)+'\r\n')

In [ ]:
file_path6 = os.path.join(results_dir, 'f1_base.txt')

with open(file_path6, 'w') as file6:
    
    file6.write(cur_electrode_name + ':' + str(f1_base)+'\r\n')